# TP I : Descentes de Gradient

Ce TP vise à apporter les éléments nécessaires pour comprendre les implementations des descentes de gradients, du moins dans un premier cadre naïf. **C'est un *TP à trous* ; il s'agira de compléter ces trous pour un rendu au plus tard le dimanche 30 août 2020.**

Voici un aperçu des points abordés lors de ce TP.

- Un set de fonctions tests: comment se donne-t-on une fonction numérique? 
- Calculer le gradient d'une fonction: coder une solution en dur ou calcul approché?
- La classe GD: une classe qui sert d'interface pour toutes les descentes qu'on souhaite implémenter. 
- La Descente de Gradient :
    - Calcul du pas : pas constant, *Backtracking*
    - Choix de norme : la classique, la norme $1$
    - Sensibilité aux points initiaux
    - Sensibilité aux nombres de conditionnement
    - Accélération : Momentum, Nesterov, Adam.
- Cas d'application classique: la régression linéaire.
- La Méthode de Newton : 
    - La classe MN
    - Comparaison à la GD.
- Face aux contraintes d'égalité.


## Attendus de rendu

Vous êtes invités à compléter ce TP et à le prolonger par le travail qui vous est suggéré. Votre rendu sera jugé à l'aune de

- votre capacité à produire des algorithmes valides, répondant à la question posée
- l'étude effectuée concernant la sensibilité de vos algorithmes aux hyperparamètres / conditions initiales
- l'analyse comparative proposée quant aux différentes implémentations suggérées 
- les stress-tests auxquels vous aurez confrontés vos implémentations. 

On portera une attention particulière à la *généricité* de votre réponse ; tout comme cela vous est suggéré par la suite on attendra de vous d'appuyer vos affirmations par suffisamment de tests et une appréciation pour les limites de votre analyse.

Ce TP est à rendre par **groupes de 3** et exceptionnellement **2**. Il sera complété d'une soutenance portant sur l'ensemble des TPs.

# Au travail!

In [ ]:
import warnings
import math
import numpy as np

## Un set de fonctions tests

Il y a différentes facons de se donner une fonction numérique en machine :

- Symboliquement : on se donne une expression symbolique représentant des sommes, différences, produits, quotients et composées de fonctions usuelles.  Dans ce cadre, l'opération de dérivation est symbolique et les dérivées obtenues par ce biais sont exactes. Une opération d'évaluation est nécessaire pour passer de l'écriture symbolique d'une fonction au calcul de la valeur qu'elle prend en un point. Vous pouvez vous attarder sur la bibliothèque `sympy` sous python, pour plus de détails.

- Numériquement : on envisage une fonction au sens algorithmique du terme, la particularité étant qu'elle a des entrées `floats` et une sortie du même type. Des approximations des fonctions usuelles sont implémentées dans les bibliothèques `math` et `numpy` de python. Dans un soucis de vectorialisation des opérations mathématiques, c'est la seconde bibliothèque qui est utilisée. Évaluer une fonction en un point consiste à appeler celle-ci avec les bonnes entrées flottantes. La dérivée d'une fonction en ce sens est une fonction au même sens qui s'exrime pour une entrée donnée à l'aide d'appels à la fonction de départ. 

- Par approximation : on se préoccupe nullement de la valeur de la fonction en tout point, il nous suffit d'avoir une liste de couples (entrée, sortie) ou encore (point, image) composées de flottants. Dans ce cas on approche les dérivées par les tangentes des droites joignants deux points successifs. 

Dans ce TP notre choix s'arrête sur la plus intuitives des démarches ; l'aspect *fonction au sens informatique du terme*. Vous êtes invités, si vous le souhaitez, à explorer les autres représentations et éventuellement les comparer à celui-ci.

### Structure de données

On va se donner un ensemble de familles de fonctions ; une famille de fonctions est donnée par une classe qui wrap les attributs:

- `name` : une `str` qui contient le nom de la fonction. 
- `convex` : `bool` spécifiant si la famille de fonction est convexe ou non.
- `dim` : `int` la dimension de l'espace de départ.
- `nb_params` : nombre de paramètres de la famille.
- `value` : une fonction qui prend en entrée un `numpy` array et renvoie un `float` en sortie.
- `grad` : fonction gradient de `function`.
- `hess` : fonction hessienne de `function`.
- `HCN` : fonction nombre de conditionnement de la hessienne de `function`.


In [ ]:
class test_family():
    
    def __init__(self, name, value, dim, nb_params, convex=None, grad=None, hess=None, HCN=None):
        self.name = name
        self.value = value
        self.dim = dim
        self.nb_params = nb_params
        self.convex = convex
        self.grad = grad
        self.hess = hess
        self.HCN = HCN

### Fonctions du set de test

In [ ]:
from matplotlib import pyplot as plt
import seaborn as sns

In [ ]:
sns.set_style("whitegrid")

1. Définir des familles de fonctions *convexes* sur $\mathbb{R}$ ayant un nombre de conditionnement uniquement dépendant des paramètres de la famille.  

In [ ]:
quad_I_d = {
    "name": "quadratic_1",
    "dim" : 1,
    "nb_params" : 1,
    "convex" : True,
    "value" : (lambda x, gamma: gamma*(x**2) + x + 1),
    "grad"  : (lambda x, gamma: 2*gamma*x + 1),
    "hess"  : (lambda x, gamma: 2*gamma),
    "HCN"   : (lambda x, gamma: 1 ) 
}

quad_I = test_family(**quad_I_d)

In [ ]:
fig, ax = plt.subplots(figsize=(16, 9))
x = np.linspace(-10, 10, 400)
ax.set_ylim(-1, 50)
for gamma in range(5, 50, 5):
    ax.plot(x, quad_I.value(x, gamma), label="gamma: {}".format(gamma))
ax.set_title("Famille quadratiques en dimension 1")
ax.legend()

In [ ]:
cubic_I_d = { 
    "name": "cubic_1",
    "dim" : 1,
    "nb_params" : 1,
    "convex" : False,
    "value" : (lambda x, gamma: x**3 + gamma*x**2 + x + 1),
    "grad"  : (lambda x, gamma: 3*x**2 + 2*gamma*x + 1),
    "hess"  : (lambda x, gamma: 6*x + 2*gamma),
    "HCN"   : (lambda x, gamma: 1 ) 
}

cubic_I = test_family(**cubic_I_d)

In [ ]:
fig, ax = plt.subplots(figsize=(16, 9))
x = np.linspace(-10, 10, 400)
ax.set_ylim(-1, 100)
for gamma in range(0, 11, 1):
    ax.plot(x, cubic_I.value(x, gamma), label="gamma: {}".format(gamma))
ax.set_title("Famille cubique en dimension 1")
ax.legend()

In [ ]:
exp_I_d = {
    "name": "exp_1",
    "dim" : 1,
    "nb_params" : 1,
    "convex" : True ,
    "value" : (lambda x, gamma: np.exp(gamma*x) + np.exp(-gamma*x)),
    "grad"  : (lambda x, gamma: gamma*(np.exp(gamma*x) - np.exp(-gamma*x))),
    "hess"  : (lambda x, gamma: (gamma**2)*(np.exp(gamma*x) + np.exp(-gamma*x))),
    "HCN"   : (lambda x, gamma: 1 )    
}

exp_I = test_family(**exp_I_d)

In [ ]:
fig, ax = plt.subplots(figsize=(16, 9))
x = np.linspace(-10, 10, 400)
ax.set_ylim(-1, 100)
params = np.linspace(0, 5, 20)
for gamma in params:
    ax.plot(x, exp_I.value(x, gamma), label="gamma: {}".format(gamma))
ax.set_title("Famille en cosinus hyperbolique en dimension 1")
ax.legend()

In [ ]:
multi_sinks_d = {
    "name": "multi_sinks_1",
    "dim" : 1,
    "nb_params" : 1,
    "convex" : False ,
    "value" : (lambda x, gamma: 20*np.cos(x**2) + (gamma * x**2)),
    "grad"  : (lambda x, gamma: np.sin(x) + 2*gamma*x),
    "hess"  : (lambda x, gamma: np.cos(x) + 2*gamma),
    "HCN"   : (lambda x, gamma: 1 )  
}

multi_sinks = test_family(**multi_sinks_d)

In [ ]:
fig, ax = plt.subplots(figsize=(16, 9))
x = np.linspace(-10, 10, 400)
ax.set_ylim(-100, 1000)
for gamma in range(1, 11, 1):
    ax.plot(x, multi_sinks.value(x, gamma), label="gamma: {}".format(gamma))
ax.set_title("Famille multi-puits en dimension 1")
ax.legend()

2. Faire de même avec des fonctions sur $\mathbb{R}^2$.

In [ ]:
quad_II_d = {
    "name": "quadric_2",
    "dim" : 2,
    "nb_params" : 1,
    "convex" : True ,
    "value" : (lambda x, gamma: x[:, 0]**2 + gamma*x[:, 1]**2),
    "grad"  : (lambda x, gamma: np.array([2*x[:, 0], 2*gamma*x[:, 1]]).reshape(1, -1)),
    "hess"  : (lambda x, gamma: np.diag([2, 2*gamma])),
    "HCN"   : (lambda x, gamma: gamma)
}

quad_II = test_family(**quad_II_d)

In [ ]:
quad_II_d["grad"](np.array([[5, 5]]), 1)

In [ ]:
nb_pts, x_mi, x_ma, y_mi, y_ma = 200, -2, 2, -2, 2
fig, ax = plt.subplots(2, 1, figsize=(20, 40))
x, y = np.linspace(x_mi, x_ma, nb_pts), np.linspace(x_mi, x_ma, nb_pts)
X, Y = np.meshgrid(x, y)
## Reshaping X, Y for proper evaluation by input function
x_y = np.vstack([X.reshape(1, -1), Y.reshape(1, -1)]).reshape(1, 2, -1)
for i in [0, 1]:
    ax[i].set_xlim(x_mi, x_ma)
    ax[i].set_ylim(y_mi, y_ma)
ax[0].contour(X, Y, quad_II.value(x_y, 10).reshape(nb_pts, -1), 15)
ax[1].contour(X, Y, quad_II.value(x_y, 1).reshape(nb_pts, -1), 15)

3. Construire des familles de fonctions tests de dimensions de plus en plus grandes en fonction du paramètre. 

4. Construire une `test_list` contenant la liste des familles de fonctions tests crées jusque là.

In [ ]:
test_list = [quad_I, cubic_I, exp_I, multi_sinks, quad_II]

## Différencier une fonction

Pour calculer le gradient d'une fonction on a déjà besoin de savoir calculer la dérivée d'une fonction réelle. On s'intéresse à cette question dans cette section. Le point central est le fait qu'approcher numériquement la dérivée d'une fonction réelle est *prone* aux erreurs numériques ; on est souvent dans une meilleure posture lorsque l'on a une expression explicite pour le gradient d'une fonction.

### L'approche naïve

On rappelle que pour une fonction numérique le nombre dérivée d'une fonction $f : \mathbb{R} \to \mathbb{R}$ en un point $x \in \mathbb{R}$ est donnée par la limite :

$$ f'(x) = \lim_{h \to 0} \frac{f(x+h)-f(x)}{h} $$

quand celle-ci existe. D'un point de vue numérique une assez petite valeur $h$ donne un quotient suffisamment proche du nombre dérivée que l'on cherche. Ainsi pour $h$ assez petit on approche $f'(x)$ par 

$$ f'(x) \simeq \frac{f(x + h) - f(x)}{h} .$$

L'erreur à l'approximation est en $o(1)$ en $0$. 

1. Coder une fonction `partial_` qui calcule la dérivée partielle d'une fonction à plusieurs variables en utilisant la démarche précédente.

In [ ]:
def partial_(f, x, i=0, dx=1e-6):
    """Computes i-th partial derivative of f at point x.
    
    Args:
        f: objective function.
        x: point at which partial derivative is computed.
        i: coordinate along which derivative is computed.
        dx: slack for finite difference.
        
    Output:
        (float)

    """
    x = x.reshape(1, -1)
    h = np.zeros(x.shape)
    h[0, i] = dx
    return (f(x + h) - f(x)) / dx

2. Tester votre fonction sur un exmple dont vous connaissez la dérivée partielle et comparer vos résutlats.

In [ ]:
partial_(lambda x: np.exp(x), np.array([100]))

In [ ]:
np.exp(np.array([100]))

**La différence est *petite* ou *grande*?**

### Une démarche un peu moins naïve

On peut en réalité faire un peu mieux ; le nombre dérivée est également la limite 

$$ f'(x) = \lim_{h \to 0} \frac{f(x+h)-f(x-h)}{2h} $$.

On approche donc $f'(x)$ par 

$$f'(x) \simeq \frac{f(x+h)-f(x-h)}{2h}$$.

La différence avec la première approximation résulte de l'écriture du DL du numérateur de cette seconde écriture (en supposant $f$ deux fois différentiable:

$$f(x + h) - f(x - h) = (f(x) + hf'(x) + \frac{h^2}{2}f''(x) + o(h^2)) - (f(x) - hf'(x) + \frac{h^2}{2} f''(x) + o(h^2))$$

Et donc:

$$f(x + h) - f(x - h) = 2hf'(x) + o(h^2)$$.

On trouve que l'erreur d'approximation dans le second cas est désormais $o(h)$ ; ce qui est supposé être une meilleure approximation que dans le premier cas.

**Remarque : si l'on souhaite faire un calcul précis de l'erreur dans l'approximation de la dérivée de $f$ en un point on s'y prendrait pas tout à fait comme cela, nous ne donnons ici qu'une vague idée de la différence entre les deux démarches précédentes.**

1. Utiliser la démarche précédente pour approcher la dérivée partielle d'une fonction en un point. Cette fonction sera notée `partial`.

In [ ]:
def partial(f, x, i=0, dx=1e-6):
    """Computes i-th partial derivative of f at point x.
    
    Args:
        f: objective function.
        x: point at which partial derivative is computed.
        i: coordinate along which derivative is computed.
        dx: slack for finite difference.
        
    Output:
        (float)

    """
    x = x.reshape(1, -1)
    h = np.zeros(x.shape)
    h[0, i] = dx
    return (f(x + h) - f(x - h)) / (2*dx)

2. Comparer `partial` à `partial_` et à l'expression exacte de la dérivée partielle d'une fonction de votre choix. 

In [ ]:
partial(lambda x: np.exp(x), np.array([100]))

In [ ]:
np.exp(np.array([100]))

Il est possible de raffiner d'avantage l'estimation de la dérivée d'une fonction en un point. On se contente de ces deux premières approches simples. Pour plus de détail vous pouvez vous référer au cours d'Analyse numérique de J-P. Demailly.

4. Effectuer des tests plus larges sur les divergences des différents schémas d'approximation précédents.

### Le calcul du gradient

On sait déjà par la démarche précédente que l'approximation numérique de la dérivée partielle d'une fonction en un point sera difficilement meilleure qu'une expression exacte. Parfois on n'est pas en mesure de trouver facilement et à la main la dérivée partielle d'une fonction ; on se garde donc la possibilité de calculer numériquement le gradient d'une fonction.

- Écrire une fonction `gradient` qui renvoie le gradient d'une fonction en un point.

In [ ]:
def gradient(f, x, dx=1e-6):
    """Computes gradient of f at point x.
    
    Args:
        f: objective function.
        x: point at which gradient is computed.
        dx: slack for finite difference of partial derivatives.
        
    Output:
        (ndarray) of size domain of f.
        
    """
    x = x.reshape(1, -1)
    dim = x.shape[1]
    return np.array([partial(f, x, i, dx) for i in range(dim)]).reshape(1, -1)

In [ ]:
gradient(lambda x: x[0, 0]**5 + x[0, 1]*2, np.array([1, 10]))

## La classe `GD`

La classe correspond à l'interface par laquelle on va implémenter et comparer les différentes variantes des descentes de gradients. L'objectif de cette section est de remplir les différents composants de calcul du principe qu'on a vu en cours. 

In [ ]:
class GD():
    """Gradient Descent Object.
    
    Implements gradient descent aiming to compute optimal objective 
    value of convex functions and local optimal ones of none 
    convex functions.
    
    """    
    def __init__(self, ddir=None, rate=None, decay=None, tol=None, max_iter=None):
        """        
        Instantiates a GD object.
    
        Attributes:
        ddir: function computing descent direction.
        rate: function computing learning rate ; takes in
              - x (ndarray): current iterate
              - f (function): objective function
              - dir_x (ndarray) : descent direction
              - grad (function) : gradient function
              - nb_iter (int): number of iterations.
              - eta (float): hyper-parameter.
        decay: function computing decay.
        tol: slack tolerance.
        max_iter: upper bound on number of iterations.
    
        """
        self.ddir = ddir if ddir else (lambda x, f, grad, tol: - grad(x, f, tol))
        self.rate = rate if rate else (lambda x, f, dir_x, grad, nb_iter, eta : 0.01)
        self.decay = decay if decay else (lambda x, f, grad, tol: np.linalg.norm(grad(x, f, tol)))
        self.tol = tol if tol else 1e-6
        self.max_iter = max_iter if max_iter else 1000
    
    def __call__(self, x, f, grad, verbose=False):
        """Calling gradient descent object with specific starting point and optimal function.
        
        Args:
            x: initial starting point for descent.
            f: objective function of optimisation problem.
            grad: function outputing gradient value of function f at a given point.
        
        Output:
            (float) sub-optimal value up to tolerance if execution is proper.
            (ndarray) list of gradient descent iterates.
            (ndarray) list of graidents of iterates.
            (int) number of iterations.
            
        """
        x = x.reshape(1, -1)
        eta = 0
        n_iter = 0

        dir_x = self.ddir(x, f, grad, self.tol)
        rate_ = self.rate(x, f, dir_x, grad, n_iter, eta)
        delta_x = rate_ * dir_x
        iters, iters_dir = x, delta_x
        
        decay_x = self.decay(x, f, grad, self.tol)
        while decay_x > self.tol and n_iter < self.max_iter:
            ## Deciding on direction
            dir_x = self.ddir(x, f, grad, self.tol)
            rate_ = self.rate(x, f, dir_x, grad, n_iter, eta)
            delta_x =  rate_ * dir_x
            
            # Storing iterates
            iters = np.vstack([iters, x])
            
            ## Updating iterate
            x = x + delta_x
            
            ## Storing directions
            iters_dir = np.vstack([iters_dir, delta_x])
            
            ## Computing decay
            decay_x = self.decay(x, f, grad, self.tol)
            
            ## Updating iteration number
            n_iter += 1
            
        if decay_x > self.tol:
            warnings.warn("Decay didn't get under tolerance rate.", RuntimeWarning)
        
        if verbose:
            msg = " Iteration nu. = {}\n approx. = {}\n ob value = {}\n and decay = {}."
            print(msg.format(n_iter, x.flatten(), f(x), decay_x))
        
        return (x, iters, iters_dir, n_iter) 

### GD dans le cas d'un pas constant

Pour être en mesure de faire nos premiers tests d'utilisation de `GD` il nous faut encore se préoccuper des quelques hyperparamètres à fixer. On commence en un premier temps par tester la descente de gradient classique à pas constants.
 

1. Mettre les valeurs par défaut pour les attributs de `GD` de facon à obtenir une descente de gradients classique à pas constant et une condition d'arrêt donnée par le fait que le gradient en un point tombe en-deça d'un certain seuil de tolérance.

*Fait pour vous dans la GD, plus haut.*

2. Tester cette descente de gradient classique sur les différentes fonctions tests. Répertorier dans un tableau le point initial de chaque descente, le nombre d'itérations, le pas choisi et la valeur objectif obtenue.

Cherchons d'abord à manipuler la classe qu'on vient de construire

In [ ]:
GD_default = GD()

In [ ]:
op_pt, iters, iters_dir, n_iter = GD_default(np.array([10]), (lambda x : quad_I.value (x, 2)), 
                                             (lambda x, f, tol : quad_I.grad (x, 2)), True)

On peut visualiser dans lors de l'optimisation de problèmes de petites dimensions les itérés de la descente. `matplotlib` est là pour  vous aider dans votre démarche.

In [ ]:
fig, ax = plt.subplots(figsize=(16, 10))
f = lambda x: quad_I.value (x, 1)
x = np.linspace(-10, 10, 400)
ax.set_ylim(-50, 400)
ax.plot(x, f(x), "k-", label="Objective function")
# reshaping iters for input
iters_reshape = iters.reshape(iters.shape[0], 1)
ax.plot(iters_reshape, f(iters_reshape), 'r.', label="Iterates")
ax.quiver(iters_reshape, f(iters_reshape), iters_dir, -1, 
          color='r', scale=None, width=0.002, headwidth=5, headlength=10)
ax.set_title("Gradient Descent in dimensin 1.")
ax.legend()

On se contente ici de travailler avec les fonctions tests de dimension $1$. **À vous de généraliser.**

In [ ]:
test_list_d1 = [test_family for test_family in test_list if test_family.dim == 1]

In [ ]:
init_pt = np.array([1])

In [ ]:
gamma = 1

In [ ]:
cst_rate = 0.02

In [ ]:
import pandas as pd

In [ ]:
GD_test_summary = pd.DataFrame(columns=["init_pt", "rate", "optimal_pt", "nb_iter"])
GD_list = {}
for t_function in test_list_d1:
    idt = 0
    GD_list[t_function.name] = GD(rate = lambda x, f, dir_x, grad, nb_iter, eta : cst_rate)
    opt_pt, _, _, n_iter = GD_list[t_function.name](init_pt, 
                                                    (lambda x: t_function.value (x, gamma)), 
                                                    (lambda x, f, tol : t_function.grad(x, gamma)))
    naming_format = "{}_{}".format(t_function.name, idt)
    GD_test_summary.loc[naming_format] = pd.Series({"init_pt": init_pt, 
                                                     "rate": GD_list[t_function.name].rate (*[None]*6), 
                                                     "optimal_pt": opt_pt, 
                                                     "nb_iter": n_iter})
    idt += 1
GD_test_summary

3. Étudier pour un sous-ensemble de votre choix la relation entre le pas de la descente et le nombre d'itérations. Prendre soins de bien vérifier que vous avez convergence.

In [ ]:
init_pt = np.array([10])

In [ ]:
gammas = np.linspace(1, 10, 10)

In [ ]:
cst_rates = np.linspace(0.01, 0.5, 200)

In [ ]:
GD_rate_iter = pd.DataFrame(columns=["gamma", "rate", "optimal_pt", "optimal_val", "nb_iter"])
GD_list_ri = {}
for t_function in test_list_d1:
    for gamma in gammas:
        idt = 0
        for cst_rate in cst_rates:
            GD_list_ri[t_function.name] = GD(rate = lambda x, f, dir_x, grad, nb_iter, eta : cst_rate)
            opt_pt, _, _, n_iter = GD_list_ri[t_function.name](init_pt, 
                                                               (lambda x: t_function.value (x, gamma)), 
                                                               (lambda x, f, tol : t_function.grad(x, gamma)))
            naming_format = "{}_{}_{}".format(t_function.name, idt, gamma)
            GD_rate_iter.loc[naming_format] = pd.Series({"gamma": gamma, 
                                                         "rate": GD_list_ri[t_function.name].rate (*[None]*6), 
                                                         "optimal_pt": opt_pt,
                                                         "optimal_val": t_function.value(opt_pt, gamma),
                                                         "nb_iter": n_iter})
            idt += 1

In [ ]:
list_of_test_types = ["quadratic", "cubic", "exp", "multisinks"]

In [ ]:
nb_figures = 2*len(gammas)
fig, ax = plt.subplots(nb_figures, 1, figsize=(16, 5*nb_figures))
selection = GD_rate_iter.index.str.contains("quadratic", regex=False)
i = 0
for gamma in gammas:
    df_selected = GD_rate_iter[(selection) & (GD_rate_iter["gamma"] == gamma)]
    ax[i].plot(df_selected["rate"], df_selected["nb_iter"])
    ax[i].set_xlim(0, 0.5)
    ax[i+1].plot(df_selected["rate"], df_selected["optimal_val"], "r")
    ax[i+1].set_ylim(-1, 100)
    ax[i+1].set_xlim(0, 0.5)
    ax[i].set_title("Gradient Descent in dimensin 1 for gamma = {}.".format(gamma))
    ax[i].legend()
    i += 2

4. Étudier la relation entre la vitesse de convergence et le point initial de votre descente dans le cas des fonctions numériques.

### GD dans le cas de *backtracking*

Vous devriez avoir constatés que le choix du pas de descente dans le cas constant est crucial pour garantir la convergence de l'algorithme de descente. Dans cette section on s'intéresse à un calcul adaptatif du pas de descente qui permet de mieux garantir la convergence de notre algo. Le désavatage est le temps que prend désormais chaque itération pour s'exécuter.

1. Écrire une fonction `backtracking` qui permet de calculer le pas par *backtracking* à une itération donnée. Pour rappel le *backtracking* a deux hyper-paramètre $\alpha$ et $\beta$ respectivement mis par défaut à $0.01$ et $0.08$.

In [ ]:
class backtracking():
    
    def __init__(self, alpha=0.01, beta=0.8, max_iter=100):
        self.alpha = alpha
        self.beta = beta
        self.max_iter = max_iter
        
    def __call__(self, x, f, dir_x, grad, n_iter, eta, tol=1e-6):
        n_while = 0
        t = 1
        x = x.reshape(1, -1)
        grad_f = grad(x, f, tol)
        while f(x + t * dir_x) > f(x) + self.alpha * t * grad_f.dot(dir_x.T) and n_while < self.max_iter:
            t = self.beta * t
            n_while += 1
        return t

2. Comparer la GD par backtracking aux tests effectués jusqu'à présent dans le cas d'une GD à pas constant (à vous de réfléchir à ce que vous voulez comparer).

In [ ]:
bt_rate = backtracking()

In [ ]:
gammas

In [ ]:
GD_ri_bt = pd.DataFrame(columns=["gamma", "rate", "optimal_pt", "optimal_val", "nb_iter"])
GD_list_bt = {}
for t_function in test_list_d1:
    for gamma in gammas:
        GD_list_bt[t_function.name] = GD(rate = bt_rate)
        opt_pt, _, _, n_iter = GD_list_bt[t_function.name](init_pt, 
                                                           (lambda x: t_function.value (x, gamma)), 
                                                           (lambda x, f, tol : t_function.grad(x, gamma)))
        naming_format = "{}_{}".format(t_function.name, gamma)
        GD_ri_bt.loc[naming_format] = pd.Series({ "gamma": gamma, 
                                                  "rate" : "backtracking",
                                                  "optimal_pt": opt_pt,
                                                  "optimal_val": t_function.value(opt_pt, gamma),
                                                  "nb_iter": n_iter })
GD_ri_bt

### Changer de direction de descente

On a explicité en cours trois types de descentes : 

 - La déscente standard : la direction de descente est celle du gradient, ce qu'on vient de regarder.
 - La déscente de plus forte pente dans le cas de la norme $\ell_1$ : la direction de descente suit le vecteur de la base canonique de plus grande dérivée partielle en valeur absolue.
 - L'algorithme de Newton où il s'agit de calculer l'inverse des hessiennes au point courant. 
 
*L'algorithme de Newton sera abordé dans une section à part.* On va donc se concentrer pour commencer sur le cas de descente en norme $\ell_1$. 
 

1. Écrire une fonction `dsgd` qui calcule la direction de descente de plus forte pente dans le cas de la norme $\ell_1$. 

In [ ]:
def dsgd(x, f, grad, tol):
    x = x.reshape(1, -1)
    dim = x.shape[1]
    sdescent = np.array([0]*dim, dtype=float).reshape(1, -1)
    grad_f = grad(x, f, tol)
    abs_grad = np.abs(grad_f)
    imax_diff = np.argmax(abs_grad) 
    sdescent[0, imax_diff] = grad_f[0, imax_diff]
    return (- sdescent)      

2. Comparer la descente de gradient $\ell_1$ à la descente classique.

Effectuer une telle comparaison en dimension $1$ n'a pas de sens (pourquoi?). On fait donc une partie de l'étude précédente en dimension $2$ pour avoir un référentiel de comparaison.

In [ ]:
test_list_d2 = [test_family for test_family in test_list if test_family.dim == 2]

In [ ]:
init_pt_d2 = np.array([[2, 2]])

In [ ]:
GD_ri_2 = pd.DataFrame(columns=["gamma", "rate", "optimal_pt", "optimal_val", "nb_iter", "iters", "iters_dir"])
GD_list_2 = {}
for t_function in test_list_d2:
    for gamma in gammas:
        GD_list_2[t_function.name] = GD(rate = lambda x, f, dir_x, grad, nb_iter, eta : 0.1)
        opt_pt, iters, iters_dir, n_iter = GD_list_2[t_function.name](init_pt_d2, 
                                                     (lambda x: t_function.value (x, gamma)), 
                                                     (lambda x, f, tol : t_function.grad(x, gamma)))
        naming_format = "{}_{}".format(t_function.name, gamma)
        GD_ri_2.loc[naming_format] = pd.Series({ "gamma": gamma, 
                                                 "rate" : 0.01,
                                                 "optimal_pt": opt_pt,
                                                 "optimal_val": t_function.value(opt_pt, gamma),
                                                 "nb_iter": n_iter,
                                                 "iters": iters,
                                                 "iters_dir": iters_dir})
GD_ri_2.loc[:, "gamma": "nb_iter"]

In [ ]:
nb_pts, x_mi, x_ma, y_mi, y_ma = 200, -2, 2, -2, 2
fig, ax = plt.subplots(2, 1, figsize=(15, 30))
x, y = np.linspace(x_mi, x_ma, nb_pts), np.linspace(x_mi, x_ma, nb_pts)
X, Y = np.meshgrid(x, y)
## Reshaping X, Y for proper evaluation by input function
x_y = np.vstack([X.reshape(1, -1), Y.reshape(1, -1)]).reshape(1, 2, -1)
## Plotting level curves for extreme cases
extreme_cases = [gammas[0], gammas[-1]]
for i in range(len(extreme_cases)):
    ax[i].set_xlim(x_mi, x_ma)
    ax[i].set_ylim(y_mi, y_ma)
    ax[i].contour(X, Y, test_list_d2[0].value(x_y, gammas[i]).reshape(nb_pts, -1), 15)
    ax[i].quiver(GD_ri_2.loc["quadric_2_{}".format(gammas[i]), "iters"][:, 0], 
                 GD_ri_2.loc["quadric_2_{}".format(gammas[i]), "iters"][:, 1], 
                 GD_ri_2.loc["quadric_2_{}".format(gammas[i]), "iters_dir"][:, 0], 
                 GD_ri_2.loc["quadric_2_{}".format(gammas[i]), "iters_dir"][:, 1],
                 color='b', scale=6, width=0.002, headwidth=4, headlength=5)
    ax[i].plot(GD_ri_2.loc["quadric_2_{}".format(gammas[i]), "iters"][:, 0], 
               GD_ri_2.loc["quadric_2_{}".format(gammas[i]), "iters"][:, 1],
               'b.')

In [ ]:
GD_ri_1 = pd.DataFrame(columns=["gamma", "rate", "optimal_pt", "optimal_val", "nb_iter", "iters", "iters_dir"])
GD_list_1 = {}
for t_function in test_list_d2:
    for gamma in gammas:
        GD_list_1[t_function.name] = GD(ddir = dsgd, 
                                        rate = lambda x, f, dir_x, grad, nb_iter, eta : 0.1)
        opt_pt, iters, iters_dir, n_iter = GD_list_1[t_function.name](init_pt_d2, 
                                                     (lambda x: t_function.value (x, gamma)), 
                                                     (lambda x, f, tol : t_function.grad(x, gamma)))
        naming_format = "{}_{}".format(t_function.name, gamma)
        GD_ri_1.loc[naming_format] = pd.Series({ "gamma": gamma, 
                                                 "rate" : 0.01,
                                                 "optimal_pt": opt_pt,
                                                 "optimal_val": t_function.value(opt_pt, gamma),
                                                 "nb_iter": n_iter,
                                                 "iters": iters,
                                                 "iters_dir": iters_dir})
GD_ri_1.loc[:, "gamma": "nb_iter"]

In [ ]:
nb_pts, x_mi, x_ma, y_mi, y_ma = 200, -2, 2, -2, 2
fig, ax = plt.subplots(2, 1, figsize=(15, 30))
x, y = np.linspace(x_mi, x_ma, nb_pts), np.linspace(x_mi, x_ma, nb_pts)
X, Y = np.meshgrid(x, y)
## Reshaping X, Y for proper evaluation by input function
x_y = np.vstack([X.reshape(1, -1), Y.reshape(1, -1)]).reshape(1, 2, -1)
## Plotting level curves for extreme cases
extreme_cases = [gammas[0], gammas[-1]]
for i in range(len(extreme_cases)):
    ax[i].set_xlim(x_mi, x_ma)
    ax[i].set_ylim(y_mi, y_ma)
    ax[i].contour(X, Y, test_list_d2[0].value(x_y, gammas[i]).reshape(nb_pts, -1), 15)
    ax[i].quiver(GD_ri_1.loc["quadric_2_{}".format(gammas[i]), "iters"][:, 0], 
                 GD_ri_1.loc["quadric_2_{}".format(gammas[i]), "iters"][:, 1], 
                 GD_ri_1.loc["quadric_2_{}".format(gammas[i]), "iters_dir"][:, 0], 
                 GD_ri_1.loc["quadric_2_{}".format(gammas[i]), "iters_dir"][:, 1],
                 color='b', scale=6, width=0.002, headwidth=4, headlength=5)
    ax[i].plot(GD_ri_1.loc["quadric_2_{}".format(gammas[i]), "iters"][:, 0], 
               GD_ri_1.loc["quadric_2_{}".format(gammas[i]), "iters"][:, 1],
               'b.')

On en vient désormais à comparer ces deux algos. Vu que les mise-à-jour internes semble demander le même nombre d'itérations il est raisonnable de s'intéresser en premier temps au nombre d'itérations.

In [ ]:
fig, ax = plt.subplots(figsize=(16, 9))
ax.set_xlim(0, 11)
ax.set_ylim(-1, 1000)
ax.plot(GD_ri_2["gamma"], GD_ri_2["nb_iter"], label="l2")
ax.plot(GD_ri_1["gamma"], GD_ri_1["nb_iter"], "r", label="l1")
ax.set_title("Comparing number of iterates of l1 and l2 gradient descents against gamma values")
ax.legend()

## Sensibilité de GD aux conditionnements de la hessienne

On s'intéresse dans cette section au comportement de la descente de gradients vis-à-vis de la géométrie locale de l'itéré courant. Cette étude va vous permettre de comprendre en quoi le manque de symétrie locale des fonctions objectifs rend plus lente la convergence de la descente de gradient.

1. En utilisant la GD classique à pas constant tracer dans les cas de nombres de conditionnements qui ne dépendent que des paramètres de vos familles tests ce nombre contre le nombre d'itérations de la GD.

2. Faire de même avec le cas de *backtracking*

3. Que constatez-vous?

## Accélérations

Il y a des stratégies standards d'accélération de descente de gradients ; on en invente même tous les ans. Il n'y a pas nécessairement de relation d'ordre entre celles-ci, certaines sont plus adaptées que d'autres à des problèmes spécifiques et inversement. On propose d'en implémenter $3$ ici, il s'agit en particulier d'un travail bibliographique (simple).

1. Implémenter la *Momentum Optimisation*. Cherchez à en comprendre le sens.

2. Implémenter la *Nesterov Optimisation*. Cherchez à en comprendre le sens.

3. Implémenter la *Adam Optimisation*. Cherchez à en comprendre le sens.

4. Comparer les descentes de gradient pour chacune des optimisations précédentes.

## Méthode de Newton

1. Implémenter la méthode de Newton.

2. Qu'est-ce qu'un exemple pertinent pour comparer la méthode de Newton au méthode de descente vu précédemment?

3. Comparer la méthode de Newton aux descentes précédentes.

## Cas pratique : la régression (YAT)

Dans cette section on attend de vous que vous implémtentier une régression linéaire (puis polynomiale) à la main. Il revient à vous d'effectuer l'étude bibliographique (ou de revoir les documents à votre disposition dans le cours et / ou mon git) pour éventuellement vous aider. À vous de pousser cette étude là où vous souhaitez ; j'attends de vous que vous puissiez entrevoir le besoin de régularisation, sujet abordé en second TP.